<a href="https://colab.research.google.com/github/VinKKAP/Data-Analysis-with-LLM/blob/main/Experiment_mit_Qwen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install git


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [2]:
!git clone https://github.com/VinKKAP/Data-Analysis-with-LLM.git

Cloning into 'Data-Analysis-with-LLM'...
remote: Enumerating objects: 236, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 236 (delta 6), reused 9 (delta 2), pack-reused 215 (from 1)
Receiving objects: 100% (236/236), 57.47 MiB | 16.00 MiB/s, done.
Resolving deltas: 100% (75/75), done.


In [ ]:
!pip install -r /content/Data-Analysis-with-LLM/Paper/requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip install --upgrade transformers scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 106.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 91.3 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.31.0
    Uninstalling transformers-4.31.0:
      Successfully uninstalled transformers-4.31.0


In [6]:
%pip uninstall -y torch transformers
%pip install torch transformers


Found existing installation: torch 2.5.1
Uninstalling torch-2.5.1:
  Successfully uninstalled torch-2.5.1
Found existing installation: transformers 4.46.3
Uninstalling transformers-4.46.3:
  Successfully uninstalled transformers-4.46.3
Note: you may need to restart the kernel to use updated packages.


You can safely remove it manually.


  Using cached torch-2.5.1-cp311-cp311-win_amd64.whl.metadata (28 kB)
  Using cached transformers-4.46.3-py3-none-any.whl.metadata (44 kB)
Using cached torch-2.5.1-cp311-cp311-win_amd64.whl (203.1 MB)
Using cached transformers-4.46.3-py3-none-any.whl (10.0 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
print(torch.cuda.is_available())

True


In [8]:
%pip install gdown

Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
torch.cuda.empty_cache() # Removed the extra indentation


In [2]:
!nvidia-smi

Wed Dec  4 10:28:12 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.03                 Driver Version: 566.03         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1070      WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   38C    P8             16W /  230W |     862MiB /   8192MiB |     14%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
import torch
print(torch.cuda.is_available())

False


In [ ]:
'''
Created on Aug 12, 2023

@author: immanueltrummer
'''
from multiprocessing import set_start_method
try:
    set_start_method('spawn')
except RuntimeError:
    pass

import argparse
import sklearn.metrics as metrics
import pandas as pd
import random as rand
import time
import torch

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

from google.colab import drive
drive.mount('/content/drive')
from google.colab import userdata
userdata.get('HF_TOKEN')

def add_type(row):
    """ Enrich column name by adding column type.

    Args:
        row: describes correlation between two columns.

    Returns:
        row with enriched column names.
    """
    row['column1'] = row['column1'] + ' ' + row['type1']
    row['column2'] = row['column2'] + ' ' + row['type2']
    return row

def def_split(data, test_ratio, seed):
    """ Split data into training and test set.

    With this approach, different column pairs from the
    same data set may appear in training and test set.

    Args:
        data: a pandas dataframe containing all data.
        test_ratio: ratio of test cases after split.
        seed: random seed for deterministic results.

    Returns:
        a tuple containing training, then test data.
    """
    print('Data sets in training and test may overlap')
    x_train, x_test, y_train, y_test = train_test_split(
      data[['column1', 'column2', 'type1', 'type2']], data['label'],
      test_size=test_ratio, random_state=seed)
    train = pd.concat([x_train, y_train], axis=1)
    test = pd.concat([x_test, y_test], axis=1)
    print(f'train shape: {train.shape}')
    print(f'test shape: {test.shape}')
    return train, test


def ds_split(data, test_ratio):
    """ Split column pairs into training and test samples.

    With this method, training and test set contain columns
    of disjunct data sets, making prediction a bit harder.

    Args:
        data: a pandas dataframe containing all data.
        test_ratio: ratio of test cases after splitting.

    Returns:
        a tuple containing training, then test set.
    """
    print('Separating training and test sets by data')
    counts = data['dataid'].value_counts()
    print(f'Counts: {counts}')
    print(f'Count.index: {counts.index}')
    print(f'Count.index.values: {counts.index.values}')
    print(f'counts.shape: {counts.shape}')
    print(f'counts.iloc[0]: {counts.iloc[0]}')
    nr_vals = len(counts)
    nr_test_ds = int(nr_vals * test_ratio)
    print(f'Nr. test data sets: {nr_test_ds}')
    ds_ids = counts.index.values.tolist()
    print(type(ds_ids))
    print(ds_ids)
    test_ds = rand.sample(ds_ids, nr_test_ds)
    print(f'TestDS: {test_ds}')

    def is_test(row):
        if row['dataid'] in test_ds:
            return True
        else:
            return False

    data['istest'] = data.apply(is_test, axis=1)
    train = data[data['istest'] == False]
    test = data[data['istest'] == True]
    print(f'train.shape: {train.shape}')
    print(f'test.shape: {test.shape}')
    print(train)
    print(test)
    return train[
        ['column1', 'column2', 'type1', 'type2', 'label']], test[
            ['column1', 'column2', 'type1', 'type2', 'label']]


def baseline(col_pairs):
    """ A simple baseline predicting correlation via Jaccard similarity.

    Args:
        col_pairs: list of tuples with column names.

    Returns:
        list of predictions (1 for correlation, 0 for no correlation).
    """
    predictions = []
    for cp in col_pairs:
        c1 = cp[0]
        c2= cp[1]
        s1 = set(c1.split())
        s2 = set(c2.split())
        ns1 = len(s1)
        ns2 = len(s2)
        ni = len(set.intersection(s1, s2))
        # calculate Jaccard coefficient
        jac = ni / (ns1 + ns2 - ni)
        # predict correlation if similar
        if jac > 0.5:
            predictions.append(1)
        else:
            predictions.append(0)
    return predictions


# log all metrics into summary for data subset
def log_metrics(
        coeff, min_v1, max_v2, mod_type, mod_name, scenario,
        test_ratio, sub_test, test_name, lb, ub, pred_method,
        out_path, training_time):
    """ Predicts using baseline or model, writes metrics to file.

    Args:
        coeff: predict correlation according to this coefficient.
        min_v1: lower bound on coefficient value for correlation.
        max_v2: upper bound on p-value to be considered correlated.
        mod_type: base type of language model used for prediction.
        mod_name: precise name of language model used for prediction.
        scenario: how training and test data relate to each other.
        test_ratio: ratio of column pairs used for testing (not training).
        sub_test: data frame with test cases, possibly a subset.
        test_name: write this test name into result file.
        lb: lower bound on a test-specific metric constraining test cases.
        ub: upper bound on test-specific metric, constraining test cases.
        pred_method: whether to use language model or simple baseline.
        out_path: path to result output file (results are appended).
        training_time: time taken to train the model.
    """
    sub_test.columns = [
        'text_a', 'text_b', 'type1', 'type2', 'labels', 'length', 'nrtokens']
    # print out a sample for later analysis
    print(f'Sample for test {test_name}:')
    sample = sub_test.sample(frac=0.1)
    print(sample)
    # predict correlation via baseline or model
    sub_test = sub_test[['text_a', 'text_b', 'labels']]
    samples = []
    for _, r in sub_test.iterrows():
        samples.append([r['text_a'], r['text_b']])
    s_time = time.time()
    if pred_method == 0:
        preds = baseline(samples)
    else:
        preds = model.predict(samples)[0]
    # log various performance metrics
    t_time = time.time() - s_time
    nr_samples = len(sub_test.index)
    t_per_s = float(t_time) / nr_samples
    f1 = metrics.f1_score(sub_test['labels'], preds)
    pre = metrics.precision_score(sub_test['labels'], preds)
    rec = metrics.recall_score(sub_test['labels'], preds)
    acc = metrics.accuracy_score(sub_test['labels'], preds)
    mcc = metrics.matthews_corrcoef(sub_test['labels'], preds)
    # also log to local file
    with open(out_path, 'a+') as file:
        file.write(f'{coeff},{min_v1},{max_v2},"{mod_type}",' \
                f'"{mod_name}","{scenario}",{test_ratio},' \
                f'"{test_name}",{pred_method},{lb},{ub},' \
                f'{f1},{pre},{rec},{acc},{mcc},{t_per_s},' \
                f'{training_time}\n')


def names_length(row):
    """ Calculate combined length of column names.

    Args:
        row: contains information on one column pair.

    Returns:
        combined length of column names (in characters).
    """
    return len(row['text_a']) + len(row['text_b'])

def names_tokens(row):
    """ Calculates number of tokens (separated by spaces).

    Attention: this is not the number of tokens as calculated
    by the tokenizer of the language model but an approximation.

    Args:
        row: contains information on one column pair.

    Returns:
        number of space-separated substrings in both column names.
    """
    return row['text_a'].count(' ') + row['text_b'].count(' ')


def run_experiment(src_path, coeff, min_v1, max_v2, mod_type, mod_name, scenario, test_ratio, use_types, out_path):
    # print parameters
    print(f'Coefficients: {coeff}')
    print(f'Minimal value 1: {min_v1}')
    print(f'Maximal value 2: {max_v2}')
    print(f'Model type: {mod_type}')
    print(f'Model name: {mod_name}')
    print(f'Scenario: {scenario}')
    print(f'Test ratio: {test_ratio}')

    # initialize for deterministic results
    seed = 42
    rand.seed(seed)

    # load data
    data = pd.read_csv(src_path, sep=',')
    data = data.sample(frac=1, random_state=seed)
    data.columns = [
        'dataid', 'datapath', 'nrrows', 'nrvals1', 'nrvals2',
        'type1', 'type2', 'column1', 'column2', 'method',
        'coefficient', 'pvalue', 'time']

    # enrich column names if activated
    if use_types:
        data = data.apply(add_type, axis=1)

    # Initialize the tokenizer and model from the pre-trained model name
    tokenizer = AutoTokenizer.from_pretrained(mod_name)
    model = AutoModelForSequenceClassification.from_pretrained(mod_name, num_labels=2)

    # Set a padding token if not already set
    if tokenizer.pad_token is None:
         tokenizer.pad_token = tokenizer.eos_token

    # Update the model configuration to include the pad_token_id
    model.config.pad_token_id = tokenizer.pad_token_id

    # Check if GPU is available and set the device accordingly
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # Move the model to the GPU if available
    model.to(device)

    # filter data
    data = data[data['method'] == coeff]
    nr_total = len(data.index)
    print(f'Nr. samples: {nr_total}')
    print('Sample from filtered data:')
    print(data.head())

    # label data
    def coefficient_label(row):
        """ Label column pair as correlated or uncorrelated.

        Args:
            row: describes correlation between column pair.

        Returns:
            1 if correlated, 0 if not correlated.
        """
        if abs(row['coefficient']) >= min_v1 and abs(row['pvalue']) <= max_v2:
            return 1
        else:
            return 0
    data['label'] = data.apply(coefficient_label, axis=1)

    # split into test and training
    if scenario == 'defsep':
        train, test = def_split(data, test_ratio, seed)
    elif scenario == 'datasep':
        train, test = ds_split(data, test_ratio)
    else:
        raise ValueError(f'Undefined scenario: {scenario}')

    train.columns = ['text_a', 'text_b', 'type1', 'type2', 'labels']
    test.columns = ['text_a', 'text_b', 'type1', 'type2', 'labels']
    print(train.head())
    print(test.head())

    # prepare dataset for transformers
    train_encodings = tokenizer(train['text_a'].tolist(), train['text_b'].tolist(), truncation=True, padding=True)
    test_encodings = tokenizer(test['text_a'].tolist(), test['text_b'].tolist(), truncation=True, padding=True)

    class Dataset(torch.utils.data.Dataset):
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels

        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            item['labels'] = torch.tensor(self.labels[idx])
            return item

        def __len__(self):
            return len(self.labels)

    train_dataset = Dataset(train_encodings, train['labels'].tolist())
    test_dataset = Dataset(test_encodings, test['labels'].tolist())

    # prepare training arguments
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=5,
        per_device_train_batch_size=2,  # Reduce batch size
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=8,  # Maintain effective batch size
        fp16=True,  # Enable mixed-precision training
        warmup_steps=100,  # Reduce warmup steps
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=50,  # Reduce logging frequency
        evaluation_strategy="epoch",
        optim="adamw_torch"  # Memory-efficient optimizer
        )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

    # train the model
    s_time = time.time()
    trainer.train()
    training_time = time.time() - s_time

    test['length'] = test.apply(names_length, axis=1)
    test['nrtokens'] = test.apply(names_tokens, axis=1)

    # Initialize result file
    with open(out_path, 'w') as file:
        file.write(
            'coefficient,min_v1,max_v2,mod_type,mod_name,scenario,test_ratio,'
            'test_name,pred_method,lb,ub,f1,precision,recall,accuracy,mcc,'
            'prediction_time,training_time\n')

    # use simple baseline and model for prediction
    for m in [0, 1]:
        # use entire test set (redundant - for verification)
        test_name = f'{m}-final'
        log_metrics(
            coeff, min_v1, max_v2, mod_type, mod_name, scenario,
            test_ratio, test, test_name, 0, 'inf', m, out_path, training_time)

        # test for data types
        for type1 in ['object', 'float64', 'int64', 'bool']:
            for type2 in ['object', 'float64', 'int64', 'bool']:
                sub_test = test.query(f'type1=="{type1}" and type2=="{type2}"')
                if sub_test.shape[0]:
                    test_name = f'Types{type1}-{type2}'
                    log_metrics(
                        coeff, min_v1, max_v2, mod_type, mod_name, scenario,
                        test_ratio, sub_test, test_name, -1, -1, m,
                        out_path, training_time)

        # test for different subsets
        for q in [(0, 0.25), (0.25, 0.5), (0.5, 1)]:
            qlb = q[0]
            qub = q[1]
            # column name length
            lb = test['length'].quantile(qlb)
            ub = test['length'].quantile(qub)
            sub_test = test[(test['length'] >= lb) & (test['length'] <= ub)]
            test_name = f'L{m}-{qlb}-{qub}'
            log_metrics(
                coeff, min_v1, max_v2, mod_type, mod_name, scenario,
                test_ratio, sub_test, test_name, lb, ub, m, out_path, training_time)
            # number of tokens in column names
            lb = test['nrtokens'].quantile(qlb)
            ub = test['nrtokens'].quantile(qub)
            sub_test = test[(test['nrtokens'] >= lb) & (test['nrtokens'] <= ub)]
            test_name = f'N{m}-{qlb}-{qub}'
            log_metrics(
                coeff, min_v1, max_v2, mod_type, mod_name, scenario,
                test_ratio, sub_test, test_name, lb, ub, m, out_path, training_time)


def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='binary')
    acc = accuracy_score(p.label_ids, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Example usage in a Jupyter Notebook or Google Colab
args = {
    "src_path": "/content/drive/My Drive/Colab Notebooks/Liter/correlations/correlationdata.csv",
    "coeff": "pearson",
    "min_v1": 0.9,
    "max_v2": 0.05,
    "mod_type": "Qwen",
    "mod_name": "Qwen/Qwen2-0.5B-Instruct",
    "scenario": "defsep",
    "test_ratio": 0.2,
    "use_types": 1,
    "out_path": "/content/drive/My Drive/Colab Notebooks/Liter/correlations/models"
}

run_experiment(**args)

ModuleNotFoundError: No module named 'google.colab'

In [11]:
!pip freeze

absl-py==2.1.0
accelerate==1.1.1
aiohappyeyeballs==2.4.4
aiohttp==3.11.9
aiosignal==1.3.1
altair==5.5.0
asttokens==3.0.0
attrs==24.2.0
blinker==1.9.0
cachetools==5.5.0
certifi==2024.8.30
charset-normalizer==3.4.0
click==8.1.7
colorama==0.4.6
comm==0.2.2
datasets==3.1.0
debugpy==1.8.9
decorator==5.1.1
dill==0.3.8
docker-pycreds==0.4.0
executing==2.1.0
filelock==3.16.1
frozenlist==1.5.0
fsspec==2024.9.0
gitdb==4.0.11
GitPython==3.1.43
grpcio==1.68.1
huggingface-hub==0.26.3
idna==3.10
ipykernel==6.29.5
ipython==8.30.0
jedi==0.19.2
Jinja2==3.1.4
joblib==1.4.2
jsonschema==4.23.0
jsonschema-specifications==2024.10.1
jupyter_client==8.6.3
jupyter_core==5.7.2
Markdown==3.7
markdown-it-py==3.0.0
MarkupSafe==3.0.2
matplotlib-inline==0.1.7
mdurl==0.1.2
mpmath==1.3.0
multidict==6.1.0
multiprocess==0.70.16
narwhals==1.15.2
nest-asyncio==1.6.0
networkx==3.4.2
numpy==2.1.3
packaging==24.2
pandas==2.2.3
parso==0.8.4
pillow==11.0.0
platformdirs==4.3.6
prompt_toolkit==3.0.48
propcache==0.2.1
protobuf==5